In [1]:
import os
import json
import pickle
import collections as C
import itertools as I

from common.constants import CORE_OPTIONS
from common.pantograph.dataclasses import ProblemGenerationProcess

header = ("""
import Mathlib
import Aesop

""" + '\n'.join('set_option ' + t.replace('=', ' ') for t in CORE_OPTIONS)).strip()
print(header)

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option maxRecDepth 100000
set_option tactic.hygienic false
set_option pp.fullNames true
set_option pp.funBinderTypes true
set_option pp.piBinderTypes true


In [ ]:
with open('/home/ma-user/workspace/formal_problem_generation/data/FineLeanCorpus/raw/done_chunk_145408.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
C.Counter(type(d) for d in data), len([d for d in data if (d.formal_solution_draft or '') != ''])

In [ ]:
data_success = [d for d in data if (d.formal_solution_draft or '') != '']
print(data_success[-20].formal_solution_draft)

In [ ]:
# with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/failed.pkl', 'rb') as f:
# with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/failed-revalidated.0830.pkl', 'rb') as f:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/failed-revalidated.0830.v2.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
stmt_failed_to_validate = []
for vv in data:
    if (vv.formal_solution_draft or '') != '' and json.loads(vv.metainfo).get('is_statement_validated', False) == False:
        stmt_failed_to_validate.append(vv)
print(len(stmt_failed_to_validate))

proof_failed_to_validate = []
for vv in data:
    meta = json.loads(vv.metainfo)
    if (vv.formal_solution_draft or '') != '' and meta.get('is_statement_validated', False) == True and meta.get('is_solution_validated', False) == False:
        proof_failed_to_validate.append(vv)
print(len(proof_failed_to_validate))

In [ ]:
# Statement fail: 83 -> 69
# Proof fail: 81 -> 29 -> 25

In [ ]:
for result in data:
    result : ProblemGenerationProcess
    steps = result.steps
    submission_name = steps[-1].step_code[len('submit_answer '):]
    submission_fvar_pp = [v for v in result.trajectory[-1][0] if v.name == submission_name]
    assert len(submission_fvar_pp) == 1, f'submission_name={submission_name}, new_context={[v.name for v in steps[-1].new_contexts]}'
    submission_fvar_pp = submission_fvar_pp[0]
    meta = json.loads(result.metainfo)
    if 'Tendsto (λ (n : ℕ) => f (n * x) * (Real.log (n * x) / Real.log x) * ' in result.formal_statement:
        raise

In [ ]:
open_set = set()
open_scoped_set = set()
option_set = set()
for s in result.steps:
    if s.is_introducing:
        for l in s.header.splitlines():
            l = l.strip()
            if l.endswith(' in'):
                l = l[:-len('in')].strip()
            if l.startswith('open scoped'):
                for elem in l[len('open scoped'):].strip().split():
                    open_scoped_set.add(elem)
            elif l.startswith('open'):
                for elem in l[len('open'):].strip().split():
                    open_set.add(elem)
            elif l.startswith('set_option'):
                option_set.add(l[len('set_option'):].strip())

In [ ]:
open_set

In [ ]:
open_scoped_set

In [ ]:
header = \
                    (('open scoped ' + ' '.join(open_scoped_set) + '\n') if len(open_scoped_set) > 0 else '') + \
                    (('open ' + ' '.join(open_set) + '\n') if len(open_set) > 0 else '') + \
                    '\n'.join(['set_option ' + t for t in option_set])

In [ ]:
print(header)

In [ ]:
result.header

In [ ]:
print(result.formal_statement)

In [13]:
# with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/problem_generation.20250822-005458.pkl', 'rb') as f:
# with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/problem_generation.20250828-223616.pkl', 'rb') as f:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_wg/Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack/problem_generation.20250828-210118.pkl', 'rb') as f:
    data = pickle.load(f)



In [14]:
results_all = list(data.values())

In [15]:
type(results_all[0])

common.pantograph.dataclasses.ProblemGenerationProcess

In [17]:
import msgspec
# with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack.jsonl', 'w') as f:
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/sft_ar_v2/Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack.jsonl', 'w') as f:
    for r in results_all:
        f.write(msgspec.json.encode(r).decode()+'\n')

{'informal_problem': '',
 'informal_answer': '',
 'informal_solution': '',
 'header': '',
 'formal_statement': 'example\n\n: ∃ (s : Finset (Fin 12)), (∀ i ∉ s, 0 % 2 ≠ 1 % 2) ∧ s = ∅\n:= sorry',
 'formal_solution_draft': 'have h1 {x : Fin 12 → ℤ} (h1 : ∀ i, x i ≥ 0) : ∃ s : Finset (Fin 12), ∑ i ∈ s, x i = 0 ↔ ∀ i ∈ s, x i = 0 :=\n  by\n  use ∅\n  simp\n\nhave h2 : ∃ s : Finset (Fin 12), (∀ (i : Fin 12), i ∉ s → (0 : ℤ) % 2 ≠ 1 % 2) ∧ s = ∅ :=\n  by\n  use ∅\n  simp\n\nexact h2',
 'formal_proofs': '',
 'steps': [{'step_draft': 'have h1 {x : Fin 12 → ℤ} (h1 : ∀ i, x i ≥ 0) : ∃ s : Finset (Fin 12), ∑ i ∈ s, x i = 0 ↔ ∀ i ∈ s, x i = 0 :=\n  by\n  use ∅\n  simp',
   'proof': [],
   'new_contexts': [{'t': '∀ {x : Fin 12 → ℤ}, (∀ (i : Fin 12), x i ≥ 0) → ∃ (s : Finset (Fin 12)), ∑ i ∈ s, x i = 0 ↔ ∀ i ∈ s, x i = 0',
     'v': None,
     'name': 'h1',
     'raw_name': '_uniq.253'}]},
  {'step_draft': 'have h2 : ∃ s : Finset (Fin 12), (∀ (i : Fin 12), i ∉ s → (0 : ℤ) % 2 ≠ 1 % 2) ∧ s = ∅ :=\n  

In [ ]:
results_all = C.defaultdict(list)

In [ ]:
idx_cnt = C.Counter()

In [ ]:
for ((ptype, source), idx), v in data.items():
    idx_cnt[idx] += 1
    results_all[(ptype, source)].append(v)

In [ ]:
sum([len([vv for vv in v if (vv.formal_solution_draft or '') != '']) for v in results_all.values()]), \
sum([len([vv for vv in v if (vv.formal_statement or '') != '']) for v in results_all.values()]), \
sum([len(v) for v in results_all.values()]), \
sum([json.loads(vv.metainfo)['prompt_tokens'] for v in results_all.values() for vv in v]), \
sum([json.loads(vv.metainfo)['completion_tokens'] for v in results_all.values() for vv in v])

In [ ]:
# stmt_failed_to_validate = []
# for v in results_all.values():
#     for vv in v:
#         if (vv.formal_solution_draft or '') != '' and json.loads(vv.metainfo).get('is_statement_validated', False) == False:
#             stmt_failed_to_validate.append(vv)
# print(len(stmt_failed_to_validate))

for (i, result) in enumerate(stmt_failed_to_validate):
    # with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/{ptype}.{source}.lean', 'w') as f:
    os.makedirs('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/stmt_fail', exist_ok=True)
    with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/stmt_fail/{i}.lean', 'w') as f:
        f.write(header + '\n\n')
        if result.formal_statement == '':
            raise
        meta = json.loads(result.metainfo)
        if 'original_trajectory' in meta.keys():
            meta.pop('original_trajectory')
            meta['is_reassembled'] = True
        f.write('-- ' + json.dumps(meta) + '\n')
        
        # Construct statement and proof
        f.write('-- # Problem Generation Process\nexample : False := by\n')
        for s in result.steps:
            if s.is_submitting:
                f.write('\n'.join('-- ' + l for l in s.step.splitlines() if l.strip() != '') + '\n\n')
            else:
                f.write(s.step + '\n\n')
        f.write('sorry\n\n-- # Statement & Proof\n')
        
        assert result.formal_statement.endswith(':= sorry')
        f.write('\n'.join(l + ' in' for l in result.header.splitlines()) + '\n' + result.formal_statement[:-len(':= sorry')] + ':= by\n' + '\nhave h_submission := by {\n' + '\n'.join('  ' + l for l in result.formal_solution_draft.splitlines() if l.strip() != '') + '\n}\nexact h_submission\n' + '\n\n')

In [ ]:
# proof_failed_to_validate = []
# for v in results_all.values():
#     for vv in v:
#         meta = json.loads(vv.metainfo)
#         if (vv.formal_solution_draft or '') != '' and meta.get('is_statement_validated', False) == True and meta.get('is_solution_validated', False) == False:
#             proof_failed_to_validate.append(vv)
# print(len(proof_failed_to_validate))

for (i, result) in enumerate(proof_failed_to_validate):
    # with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/{ptype}.{source}.lean', 'w') as f:
    os.makedirs('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/proof_fail', exist_ok=True)
    with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/proof_fail/{i}.lean', 'w') as f:
        f.write(header + '\n\n')
        if result.formal_statement == '':
            raise
        meta = json.loads(result.metainfo)
        if 'original_trajectory' in meta.keys():
            meta.pop('original_trajectory')
            meta['is_reassembled'] = True
        f.write('-- ' + json.dumps(meta) + '\n')
        
        # Construct statement and proof
        f.write('-- # Problem Generation Process\nexample : False := by\n')
        for s in result.steps:
            if s.is_submitting:
                f.write('\n'.join('-- ' + l for l in s.step.splitlines() if l.strip() != '') + '\n\n')
            else:
                f.write(s.step + '\n\n')
        f.write('sorry\n\n-- # Statement & Proof\n')
        
        assert result.formal_statement.endswith(':= sorry')
        f.write('\n'.join(l + ' in' for l in result.header.splitlines()) + '\n' + result.formal_statement[:-len(':= sorry')] + ':= by\n' + '\nhave h_submission := by {\n' + '\n'.join('  ' + l for l in result.formal_solution_draft.splitlines() if l.strip() != '') + '\n}\nexact h_submission\n' + '\n\n')

In [ ]:
result.formal_statement

In [ ]:
failed = stmt_failed_to_validate + proof_failed_to_validate
with open('/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/failed.pkl', 'wb') as f:
    pickle.dump(failed, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
print(vv.header)
print(vv.formal_statement)
print(vv.formal_solution_draft)

In [ ]:
for (ptype, source), ds in results_all.items():
    with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack/{ptype}.{source}.lean', 'w') as f:
        f.write(header + '\n\n')
        for result in ds:
            if result.formal_solution_draft is not None:
                assert result.formal_statement.endswith(':= sorry')
                f.write('\n'.join(l + ' in' for l in result.header.splitlines()) + '\n' + result.formal_statement[:-len(' sorry')] + ' by\n' + result.formal_solution_draft + '\n\n')

In [ ]:
for (ptype, source), ds in results_all.items():
    # with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator/{ptype}.{source}.lean', 'w') as f:
    with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/MiniF2F/Goedel-Prover-V2-8B.Numina-Lean.problem_generator.nopack/{ptype}.{source}.lean', 'w') as f:
        f.write(header + '\n\n')
        for result in ds:
            if result.formal_statement == '':
                continue
            meta = json.loads(result.metainfo)
            if 'original_trajectory' in meta.keys():
                meta.pop('original_trajectory')
                meta['is_reassembled'] = True
            f.write('-- ' + json.dumps(meta) + '\n')
            
            steps = result.steps
            submission_name = steps[-1].step[len('submit_answer '):]
            submission_fvar = [v for v in result.trajectory[-1][0] if v.name == submission_name]
            assert len(submission_fvar) == 1, f'submission_name={submission_name}, new_context={[v.name for v in steps[-1].new_contexts]}'
            submission_fvar = submission_fvar[0]
            
            # Construct statement and proof
            problem_hypotheses = []
            for s in steps:
                if s.is_introducing:
                    step_code = s.step_code
                    assert step_code.startswith('have ') and step_code.endswith(' := sorry')
                    problem_hypotheses.append('(' + step_code[len('have '):-len(' := sorry')] + ')')
            
            formal_statement = 'example\n' + '\n'.join(problem_hypotheses) + '\n: ' + submission_fvar.t + '\n:= by\n'
            f.write('\n'.join(l + ' in' for l in result.header.splitlines()) + '\n' + formal_statement + result.formal_solution_draft + '\n\n')